In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from matplotlib import cm
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import torchvision # load datasets
import torchvision.transforms as transforms # transform data
import torch.nn as nn # basic building block for neural neteorks
import torch.nn.functional as F # import convolution functions like Relu
import torch.optim as optim # optimzer
import torch.utils.data as data
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
torch.cuda.empty_cache()
import os
import glob
import gc
gc.collect()
torch.manual_seed(42)
device = torch.device("cuda:0")
# torch.set_default_tensor_type('torch.cuda.FloatTensor')
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [ ]:
tsne = TSNE(n_components=2,perplexity=20)

In [ ]:

# TRAIN_DATA_PATH = "/content/drive/MyDrive/RA/"
TRAIN_DATA_PATH = "data"
BATCH_SIZE = 4
LEARNING_RATE = 0.003
EPOCHS = 25
# mean, stf = get_mean_and_std(train_iter_loader) #We call this function after initializing the dataloader
mean = [0.8249, 0.8083, 0.8291]#calculated using the function above
std = [0.1698, 0.1917, 0.1764]#calculated using the fucntion above
TRANSFORM_IMG_train = transforms.Compose([
    transforms.Resize((512,512)),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(10),
#     transforms.RandomVerticalFlip(),
    # transforms.Resize(256),
    # transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean),torch.Tensor(std))
    ])
TRANSFORM_IMG_test = transforms.Compose([
    transforms.Resize((512,512)),
    # transforms.Resize(256),
    # transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean),torch.Tensor(std))
    ])

In [ ]:
file_list = glob.glob('data/train/'+ "*")

In [ ]:
file_list

In [ ]:
counter = 0
for folder in file_list:
    files = glob.glob( folder+ "/*.tif")
    counter+=len(files)
    

In [ ]:
counter

In [ ]:
train_iter_folder = torchvision.datasets.ImageFolder(os.path.join(TRAIN_DATA_PATH, 'train'), transform=TRANSFORM_IMG_train)
# train_iter_folder = torchvision.datasets.ImageFolder(os.path.join(TRAIN_DATA_PATH, 'train'))
train_iter_loader = DataLoader(train_iter_folder,batch_size = 100,shuffle=True,generator=torch.Generator(device='cuda'))

In [ ]:
classes = {'colon': 0,'endometrium_1': 1,'endometrium_2': 2,'kidney': 3,
'liver': 4,'lung': 5,'lymph_node': 6,'pancreas': 7,'skin_1': 8,'skin_2': 9,'small_intestine': 10,'spleen': 11}

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*125*125,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3  = nn.Linear(84,len(classes))

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*125*125)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = CNN()

In [ ]:
net.to(device)
net.cuda()

In [ ]:
net.load_state_dict(torch.load('CNS_75_RA_scratch_transform_512_trial1.pth'))

In [ ]:
# correct = 0
# total = 0
# actual = []
# predicted_list = []
# with torch.no_grad():
#     for _,(image,label) in enumerate(test_iter_loader):
#         images = image.to(device)
#         labels = label.to(device)
#         outputs = net(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         for i in range(len(predicted)):
#             actual.append(labels[i].view(-1).detach().cpu().numpy())
#             predicted_list.append(predicted[i].view(-1).detach().cpu().numpy())
# #         actual.append(labels)
# #         predicted_list.append(predicted)
#         correct += (predicted == labels).sum().item()
# # 
# print('Accuracy of the network on the test images: %d %%' % (
#     100 * correct / total))

In [ ]:
test_imgs = torch.zeros((0, 1, 512, 512),device = torch.device("cuda:0"))
test_predictions = []
test_targets = []
test_embeddings = torch.zeros((0, 200), device = torch.device("cuda:0"))

In [ ]:
for x,y in train_iter_loader:
    x = x.to(device)
    y = y.to(device)
    embeddings, logits = net(x)
    preds = torch.argmax(logits, dim=1)
    test_predictions.extend(preds.detach().cpu().tolist())
    test_targets.extend(y.detach().cpu().tolist())
    test_embeddings = torch.cat((test_embeddings, embeddings.detach().cpu()), 0)
    test_imgs = torch.cat((test_imgs, x.detach().cpu()), 0)
test_imgs = np.array(test_imgs)
test_embeddings = np.array(test_embeddings)
test_targets = np.array(test_targets)
test_predictions = np.array(test_predictions)

In [ ]:
for image,label in train_iter_loader:
    tsne_result = tsne.fit_transform(image)

In [ ]:
tsne = TSNE(2, verbose=1)
tsne_proj = tsne.fit_transform(test_embeddings)
# Plot those points as a scatter plot and label them based on the pred labels
cmap = cm.get_cmap('tab20')
fig, ax = plt.subplots(figsize=(8,8))
num_categories = 10
for lab in range(num_categories):
    indices = test_predictions==lab
    ax.scatter(tsne_proj[indices,0],tsne_proj[indices,1], c=np.array(cmap(lab)).reshape(1,4), label = lab ,alpha=0.5)
ax.legend(fontsize='large', markerscale=2)
plt.show()